In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from openpyxl import Workbook, load_workbook
#from openpyxl.utils.dataframe import dataframe_to_rows

Files = []
active = []
std = []
pop = []
IR = []
FIPS = []
counties = []
states = []

os.chdir(r'C:\Users\borde\Desktop\COVID\Daily County Reports')
for files in os.listdir(r'C:\Users\borde\Desktop\COVID\Daily County Reports'):
    Files.append(files)
end_pd = pd.read_csv(Files[-1],index_col=False)
start_pd = pd.read_csv(Files[-14], index_col=False)

end_pd = end_pd.drop(end_pd[end_pd['FIPS']>=80000].index).dropna(subset=['FIPS','Admin2', 'Active', 'Incidence_Rate', 'Province_State'])
end_pd = end_pd.sort_values(['FIPS']).reset_index()

start_pd = start_pd.drop(start_pd[start_pd['FIPS']>=80000].index).dropna(subset=['FIPS','Admin2', 'Active', 'Incidence_Rate', 'Province_State'])
start_pd = start_pd.sort_values(['FIPS']).reset_index()

for x in range(0,len(end_pd)):
    if end_pd['Incidence_Rate'][x] == 0:
        pop.append(0)
    else:
        pop.append((end_pd['Active'][x])/(end_pd['Incidence_Rate'][x])*100000)

end_pd['Population'] = pop

end_pd = end_pd.drop(['index','Country_Region','Last_Update','Lat','Long_','Confirmed','Deaths','Recovered','Combined_Key','Incidence_Rate','Case-Fatality_Ratio'],axis=1)
start_pd = start_pd.drop(['Admin2','Province_State','index','Country_Region','Last_Update','Lat','Long_','Confirmed','Deaths','Recovered','Combined_Key','Incidence_Rate','Case-Fatality_Ratio'],axis=1)
end_pd.columns = ['FIPS','Admin2','Province_State','Total','Population']
start_pd.columns = ['FIPS','14Total']

combined_pd = end_pd.merge(start_pd, on='FIPS',how='left')
combined_pd['Active'] = (combined_pd['Total'] - combined_pd['14Total']).abs()

for x in range(0,len(combined_pd)):
    if(combined_pd['Population'][x] == 0):
        IR.append(0)
    else: 
        IR.append((combined_pd['Active'][x]/combined_pd['Population'][x]*100000))
    counties.append(combined_pd['Admin2'][x])
    states.append(combined_pd['Province_State'][x])
combined_pd['Incidence_Rate'] = IR

active = combined_pd['Active']

for x in range(0,len(combined_pd)):
    FIPS.append(combined_pd['FIPS'][x].astype('int'))

std = combined_pd['Incidence_Rate'].std()
mean = combined_pd['Incidence_Rate'].mean()

medium = np.float64(mean+std)
high = np.float(mean+std*2)

os.chdir(r'C:\Users\borde\Desktop\COVID\Map Dashboard\Input')
APTDYs_pd = pd.read_csv('ApollyonData2.csv',index_col=False)
APTDYs_pd = APTDYs_pd[APTDYs_pd['County'].notna()]
APTDYs_pd['Travelers'] = 1
APTDYsGrouped_pd = APTDYs_pd.groupby(['CountyFIPS','County','State'],as_index=False).agg({'Travelers': 'sum'})

FIPS_pd = pd.DataFrame(list(zip(FIPS, counties, states, IR, active)), columns = ['FIPS', 'County', 'State', 'Incidence_Rate', 'Active'])
rates = []
risk = []

os.chdir(r'C:\Users\borde\Desktop\COVID\Risk Locator')
writer = pd.ExcelWriter('Output.xlsx', engine='xlsxwriter')
FIPS_pd.to_excel(writer, sheet_name = 'Data')
writer.save()

for x in range(0,len(APTDYsGrouped_pd)):
    rates.append(IR[FIPS.index(APTDYsGrouped_pd['CountyFIPS'][x])])

for x in range(0,len(rates)):
    if (rates[x] < medium):
        risk.append("LOW")
    elif (rates[x] >= medium) and (rates[x] < high):
        risk.append("MEDIUM")
    elif (rates[x] >= high):
        risk.append("HIGH")
    else:
        risk.append("ERROR")

APTDYsGrouped_pd['Incidence_Rate'] = rates
APTDYsGrouped_pd['Risk'] = risk

os.chdir(r'C:\Users\borde\Desktop\COVID\Risk Locator')
writer = pd.ExcelWriter('APOutput.xlsx', engine='xlsxwriter')
APTDYsGrouped_pd.to_excel(writer, sheet_name = 'Data')
writer.save()